# Домашнее задание №2

**Максимальная оценка за работу `min(X, 10)`, где X - суммарный балл за hw2 из 12 возможных.**

## Часть 1. ML workflow (**всего 5 баллов**)

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

from itertools import product

from sklearn.linear_model import Lasso

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import KFold

### Загрузим данные для работы.

In [4]:
wine_data = pd.read_csv("winequality-red.csv")
wine_data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Будем решать задачу регрессии: необходимо предсказать качество вина на основе его характеристик

### Шаг 1.  (**0.2 балла**)
Создайте матрицу X объект-признак и целевой вектор y ("quality")

In [6]:
X = wine_data.drop(columns=['quality'])
y = wine_data['quality']

X.shape, y.shape

((1599, 11), (1599,))

### Шаг 2. (**0.2 балла**)
Разбейте данные на train и test (доля тестовых данных - 30%).

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1119, 11), (480, 11), (1119,), (480,))

### Шаг 3. (**0.2 балла**)
Обучите линейную регрессию на тренировочных данных и сделайте предсказания на train и на test.

In [9]:
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

print(f'Предсказания на тренировочных данных: {y_train_pred[:5]}')
print(f'Предсказания на тестовых данных: {y_test_pred[:5]}')

Предсказания на тренировочноых данных: [6.35882698 5.9774259  5.66887262 5.67281951 5.38838989]
Предсказания на тестовых данных: [5.35676319 5.09071476 5.62553757 5.44886088 5.74478368]


### Шаг 4. (**0.4 балла**)
Выведите на экран ошибку MSE на train и на test, затем выведите на экран ошибку r2 на train и test.  

In [55]:
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f'MSE-train: {mse_train:.2f}')
print(f'MSE-test: {mse_test:.2f}')
print(f'R^2-train: {r2_train:.2f}')
print(f'R^2-test: {r2_test:.2f}')

MSE-train: 0.42
MSE-test: 0.41
R^2-train: 0.36
R^2-test: 0.35


### Шаг 5. (**0.5 балла**)
Вычислите среднее качество (r2) модели на кросс-валидации с k=5 фолдами.

In [66]:
cv_r2 = cross_val_score(model, X, y, cv=5, scoring='r2')
mean_r2 = cv_r2.mean()

print(f'Среднее качество (R^2) модели на кросс-валидации с k=5 фолдами: {mean_r2:.2f}')

Среднее качество (R^2) модели на кросс-валидации с k=5 фолдами: 0.29


### Шаг 6.  (**0.5 балла**)
Теперь примените линейную регрессию с L1-регуляризацией (Lasso) для данной задачи. Объявите модель и подберите параметр регуляризации alpha по сетке. Ищите alpha в диапазоне (0.1, 1.1) с шагом 0.1.

Осуществите подбор параметра alpha по тренировочным данным (Xtrain, ytrain).

In [15]:
alpha_variants = np.arange(0.1, 1.1, 0.1)

lasso = Lasso()
param_grid = {'alpha': alpha_variants}

grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

best_alpha = grid_search.best_params_['alpha']
best_r2_train = grid_search.best_score_

### Шаг 7.  (**0.5 балла**)
Выведите наилучший алгоритм и наилучшее качество по результатам подбора alpha (best_estimator_ и best_score_).

In [57]:
best_estimator = grid_search.best_estimator_
best_score = grid_search.best_score_

print(f'Наилучшее значение альфы: {best_estimator}')
print(f'Значение R^2 при наилучшей альфе на тренировочной выборке: {best_score:.2f}')

Наилучшее значение альфы: Lasso(alpha=0.1)
Значение R^2 при наилучшей альфе на тренировочной выборке: 0.24


### Шаг 8.  (**0.5 балла**)

С помощью найденного best_estimator_ сделайте предсказание на тестовых данных и выведите на экран r2-score на тесте.

In [58]:
y_test_pred_lasso = best_estimator.predict(X_test)

r2_test_lasso = r2_score(y_test, y_test_pred_lasso)

print(f'Значение R^2-score на тесте: {r2_test_lasso:.2f}')

Значение R^2-score на тесте: 0.21


### Шаг 9.  (**0.5 балла**)

Попробуем улучшить качество модели за счет добавления полиномиальных признаков. Создайте pipeline, состоящий из добавления полиномиальных признаков степени 2, а затем применения линейной регрессии.

Затем вычислите r2-score этой модели на кросс валидации с пятью фолдами.

In [59]:
pipeline = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('linear_regression', LinearRegression())
])

cv_r2_poly_scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')
mean_cv_r2_poly = cv_r2_poly_scores.mean()

mean_cv_r2_poly

print(f'Значение R^2: {mean_cv_r2_poly:.2f}')

Значение R^2: 0.23


### Шаг 10.  (**0.5 балла**)
Обучите модель (pipeline) на тренировочных данных и сделайте предсказания для train и test, затем выведите на экран r2-score и MSE на тренировочных и на тестовых данных.

In [60]:
pipeline.fit(X_train, y_train)

y_train_pred_poly = pipeline.predict(X_train)
y_test_pred_poly = pipeline.predict(X_test)

r2_train_poly = r2_score(y_train, y_train_pred_poly)
mse_train_poly = mean_squared_error(y_train, y_train_pred_poly)

r2_test_poly = r2_score(y_test, y_test_pred_poly)
mse_test_poly = mean_squared_error(y_test, y_test_pred_poly)

r2_train_poly, mse_train_poly, r2_test_poly, mse_test_poly

print(f'Значение R^2 для тренировочных данных: {r2_train_poly:.2f}')
print(f'Значение R^2 для тестовых данных: {r2_test_poly:.2f}')
print()
print(f'Значение MSE для тренировочных данных: {mse_train_poly:.2f}')
print(f'Значение MSE для тестовых данных: {mse_test_poly:.2f}')

Значение R^2 для тренировочных данных: 0.45
Значение R^2 для тестовых данных: 0.35

Значение MSE для тренировочных данных: 0.36
Значение MSE для тестовых данных: 0.41


### Сделайте выводы. Для этого ответьте на вопросы: (**1 балл**)

1) Хорошее ли качество показала исходная модель (линейная регрессия без регуляризации)? Является ли эта модель переобученной?

2) Помогла ли L1-регуляризация улучшить качество модели?

3) Помогло ли добавление полиномов второй степени улучшить качество модели? Как добавление новых признаков повлияло на переобучение?

**Ответы:**

1. Исходная модель показала значение R^2 около 0.36 на тренировочной выборке и 0.35 на тестовой. Получается, что объясняется треть дисперсии целевой переменной - в принципе это умеренное качество для модели линейной регрессии. Исходная модель не является переобученной - R^2 для тренировочной и тестовой выборок достаточно близки.

2. L1-регуляризация с alpha^* = 0.1 уменьшила R^2 на тестовой выборке (до 0.21) и обеспечила сжатие некоторых коэффициентов признаков к нулю. Это могло бы улучшить качество модели, если бы признаки были избыточными или имели высокую корреляцию. Однако в данном случае - линейная модель уже была достаточно простой и устойчивой к переобучению.

3. Полиномиальные признаки степени 2 увеличили выразительную мощность модели и привели к увеличению R^2 на тренировочной выборке до 0.451, но на тестовой выборке качество не изменилось. Значит, добавление полиномиальных признаков повысило способность модели объяснять дисперсию в обучающей выборке, но не дало существенного улучшения на тестовых данных. Это указывает на переобучение - полиномиальные признаки привели к увеличению сложности модели, что, по-видимому, привело к запоминанию тренировочных данных.

### *Попытайтесь улучшить модель (добейтесь наилучшего качества) - можно использовать любые методы.

При улучшении качества r2 на 0.1-0.2 +1 балл, при большем улучшении +2 балла (дополнительно к 5 баллам за основную часть).

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('random_forest', RandomForestRegressor(random_state=42))
])

param_grid_rf = {
    'random_forest__n_estimators': [100, 150],
    'random_forest__max_depth': [10, None],
}

grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='r2')
grid_search_rf.fit(X_train, y_train)

best_rf_model = grid_search_rf.best_estimator_

y_test_pred_rf = best_rf_model.predict(X_test)
r2_rf = r2_score(y_test, y_test_pred_rf)
mse_rf = mean_squared_error(y_test, y_test_pred_rf)

print(f'Значение R^2 при обучении с помощью случайного леса: {r2_rf:.3f}, MSE (test): {mse_rf:.3f}')
print(f'Результат на тестовой выборке стал лучше! - delta R^2 = {(r2_rf - r2_test_poly):.2f}')

Значение R^2 при обучении с помощью случайного леса: 0.448, MSE (test): 0.350
Результат на тестовой выборке стал лучше! - delta R^2 = 0.10


## Часть 2. Target encoding (**всего 5 баллов**)

В этом части домашнего задания вы будете работать с выборкой `1C`. Вам нужно посчитать счетчики для `item_id` четырьмя способами:

    1) При помощи KFold схемы;  
    2) При помощи Leave-one-out схемы;
    3) При помощи smoothing схемы;
    4) При помощи expanding mean схемы.

### Подготовка данных

In [32]:
sales = pd.read_csv('sales_train.csv.gz')
sales.columns = ['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'target']
sales

,date,date_block_num,shop_id,item_id,item_price,target
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [33]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'target':'sum'})

#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

### Mean encodings без регуляризации

После проделанной технической работы, мы готовы посчитать счетчики для переменной `item_id`.

Ниже приведены две реализации подсчета счетчиков без регуляризации. Можно использовать данный код в качестве стартовой точки для реализации различных техник регуляризации.

#### Способ 1

In [34]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.48303869886216966


#### Способ 2

In [35]:
'''
     Differently to `.target.mean()` function `transform`
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True)

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.48303869886216966


###  KFold схема (**1.25 балла**)

Необходимо реализовать Kfold схему с пятью фолдами. Используйте KFold(5) из sklearn.model_selection.

1. Разбейте данные на 5 фолдов при помощи `sklearn.model_selection.KFold` с параметром `shuffle=False`.
2. Проитерируйтесь по фолдам: используйте 4 обучающих фолда для подсчета средних значений таргета по `item_id` и заполните этими значениями валидационный фолд на каждой итерации.

Обратите внимание на **Способ 1** из примера. В частности, изучите, как работают функции map и pd.Series.map. Они довольно полезны во многих ситуациях.

In [54]:
kf = KFold(n_splits=5, shuffle=False)

all_data['item_target_enc_kfold'] = np.nan

for train_index, val_index in kf.split(all_data):

    train_data, val_data = all_data.iloc[train_index], all_data.iloc[val_index]

    item_id_target_mean = train_data.groupby('item_id')['target'].mean()
    
    all_data.loc[val_index, 'item_target_enc_kfold'] = val_data['item_id'].map(item_id_target_mean)

all_data['item_target_enc_kfold'].fillna(all_data['target'].mean(), inplace=True)

encoded_feature = all_data['item_target_enc_kfold'].values
correlation = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]

print(f'Корреляция: {correlation:.2f}')

Корреляция: 0.41


### Leave-one-out схема (**1.25 балла**)

Необходимо реализовать leave-one-out схему . Учтите, если вы запустите код из первого задания, задав количество фолдов такое же как размер выборки, то вы, вероятно, получите правильный ответ, но ждать будете очень-очень долго.

Для более быстрой реализации подсчета среднего таргета на всех объектах, кроме одного, вы можете:

1. Вычислить суммарный таргет по всем объектам.
2. Вычесть таргет конкретного объекта и разделить результат на `n_objects - 1`.

Заметим, что пункт `1.` следует сделать для всех объектов. Также заметим, что пункт `2.` может быть реализован без циклов `for`.

Здесь может оказаться полезной функция .transform из **Способа 2** из примера.

In [63]:
item_counts = all_data.groupby('item_id')['target'].transform('count')

item_sums = all_data.groupby('item_id')['target'].transform('sum')

all_data['item_target_enc_loo'] = (item_sums - all_data['target']) / (item_counts - 1)

all_data['item_target_enc_loo'].fillna(all_data['target'].mean(), inplace=True)

encoded_feature = all_data['item_target_enc_loo'].values
correlation = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]

print(f'Корреляция: {correlation:.2f}')

Корреляция: 0.48


### Smoothing (**1.25 балла**)

Необходимо реализовать smoothing с $\alpha = 100$. Используйте формулу:

$\frac{mean(target) \cdot nrows + globalmean \cdot \alpha }{nrows + \alpha}$,

где $globalmean=0.3343$. Заметим, что `nrows` - это количество объектов, принадлежащих конкретной категории, а не количество строк в датасете.

In [64]:
global_mean = 0.3343
alpha = 100

item_means = all_data.groupby('item_id')['target'].transform('mean')

item_counts = all_data.groupby('item_id')['target'].transform('count')

all_data['item_target_enc_smoothing'] = (item_means * item_counts + global_mean * alpha) / (item_counts + alpha)

all_data['item_target_enc_smoothing'].fillna(global_mean, inplace=True)

encoded_feature = all_data['item_target_enc_smoothing'].values
correlation = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]

print(f'Корреляция: {correlation:.2f}')

Корреляция: 0.48


### Expanding mean схема (**1.25 балла**)

Необходимо реализовать *expanding mean* схему. Ее суть заключается в том, чтобы пройти по отсортированному в определенном порядке датасету (датасет сортируется в самом начале задания) и для подсчета счетчика для строки $m$ использовать строки от $0$ до $m-1$. Вам будет необходимо воспользоваться pandas функциями [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) и [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html).

In [65]:
cumulative_sum = all_data.groupby('item_id')['target'].cumsum() - all_data['target']

cumulative_count = all_data.groupby('item_id').cumcount()

all_data['item_target_enc_expanding'] = cumulative_sum / cumulative_count

all_data['item_target_enc_expanding'].fillna(all_data['target'].mean(), inplace=True)

encoded_feature = all_data['item_target_enc_expanding'].values
correlation = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]

print(f'Корреляция: {correlation:.2f}')

Корреляция: 0.50
